# Journal Crawler
## 코드 설명
1. df : 추후 CSV파일로 내보내기 위해 사용. 구성은 다음과 같음.
    - 제목
    - Abstract
    - 저자
    - 논문지
    - 발행연도
    - 해당 논문 링크
    - 키워드
2. runpages : 어떤 논문을 인용한 논문을 수집하기 위한 링크를 3페이지 정도 돌기위해 사용
3. getAbstract : 논문의 링크에 들어가서 해당 웹페이지에서 abstract를 수집하기 위해 만들었으나, 저널별로 구조가 너무 달라 일단 보류중

## 수집방법
BFS를 사용한다. url pool에서 queue구조로 꺼내기위해 pop(0)를 사용.<br>
parameter를 분리해서 검색어를 바꿔가면서 검색할 수 있다.<br>
하나의 검색어가 들어가면 해당 결과페이지에서 4페이지까지 수집하며, \[책\]과 \[PDF\], \[인용\]은 제외한다.<br>
결과 페이지에서 `div class=gs_ri`가 논문별로 정보를 가지고 있으며 이 정보에서 위 df를 채우기 위한 정보들을 가지고 온다.<br>
위 정보 중 인용은 link이고 이 link를 가지고와서 3페이지 정도 검색하여 url pool에 추가한다. BFS를 위해 depth를 설정하였으며 depth는 3정도까지만 가지고 온다.<br>
정보를 정제하여 dictionary로 만든다음 dataframe형태로 만들어 기존의 df dataframe에 concat한다.<br>
수집이 완료되면 csv로 저장하여 db에 넣을 준비를 한다. $\leftarrow$ 이 부분은 수정이 필요.

In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
import urllib.parse 
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.DataFrame(columns=['title','abstract','author','journal','year','pdf_link','keyword'])

In [3]:
def runpages(url):
    result = list()
    pages = [0]
    for i in pages:
        newUrl = url + '&' + 'start='+ str(0+10*i)
        result.append(newUrl)
    return result

In [ ]:
## 얘는 망했어요

def getAbstract(url):
    web = get(url, headers=headers)
    print(web.headers['Content-Type'])
    dom = BeautifulSoup(web.text, 'html.parser')
    metas = dom.find_all('meta')
    for meta in metas:
        if 'property' in meta.attrs.keys():
            print(meta.attrs['property'])
#         if meta.attrs['property'] != None:
# #             print(meta)
#     print(dom)
#     cnt = 0
#     for el in dom.find_all('div'):
#         print(el.text.strip())
#         if 'abstract' == el.text.lower():
#             print(el)
#             classInfo = el.attrs['class'][0]
#     newel = dom.find('div',attrs={'class':classInfo}).find_parent()
#     for el in newel:
#         if 'abstract' not in el.text.lower():
#             abstract = el.text
#     return abstract
url = 'https://ieeexplore.ieee.org/abstract/document/1456701'

# getAbstract(url)

In [4]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

In [5]:
baseurl = 'https://scholar.google.com/scholar'
params = {
    'hl':'ko',
    'start':0,
    'as_sdt':'0%2C5',
    'q':'harmonic analysis',
    }

urls = list() # url담을 list
visited = list()
for i in [0,1]:
    params['start'] = 0+10*i
    url = baseurl + '?' + urllib.parse.urlencode(params)
    urls.append((url,0))

while urls:
    seed = urls.pop(0)
    print(seed[0])
    visited.append(seed[0])
    
    if seed[1] > 2:
        continue
        
    resp = get(seed[0], headers=headers)

    if resp.status_code != 200:
        print(resp.headers)
        print(resp.status_code)
        break
        
    else:
        dom = BeautifulSoup(resp.text, 'html.parser')
        
        # 정보 뭉치 찾기
        infos= dom.find_all('div', attrs={'class':'gs_ri'})

        for info in infos:
            isbook = info.find('span',attrs={'class':'gs_ctc'})
            iscite = info.find('span',attrs={'class':'gs_ctu'})
            if isbook != None and 'B' in isbook.text or iscite != None:
                continue
            else:
                data = dict()
                _ = info.find('h3',attrs={'class':'gs_rt'}).find('a')
                citing = info.find('div',attrs={'class':'gs_fl'})
                aTags = citing.find_all('a')
                citeLink = 'https://scholar.google.com' + aTags[2].attrs['href']
                citeLinks = runpages(citeLink)

                for l in citeLinks:
                    urls.append((l,seed[1]+1))
                
                link = _.attrs['href']
                data['pdf_link'] = [link]
                title = _.text
                data['title'] = [title]

                ajy = info.find('div',attrs={'class':'gs_a'}).text.split('-')

                author = ajy[0].strip()
                data['author'] = [author]
                journal = ajy[1].strip()[:-4].split('\xa0')[0]
                data['journal'] = [journal]
                year = ajy[1].strip()[-4:]
                data['year'] = [year]
                data['abstract'] = ['']
                data = pd.DataFrame(data)
                df = pd.concat([df,data])
    print(len(urls),len(visited))
    time.sleep(10)
    

https://scholar.google.com/scholar?hl=ko&start=0&as_sdt=0%252C5&q=harmonic+analysis
4 1
https://scholar.google.com/scholar?hl=ko&start=10&as_sdt=0%252C5&q=harmonic+analysis
11 2
https://scholar.google.com/scholar?cites=5849802911125373081&as_sdt=2005&sciodt=0,5&hl=ko&start=0
16 3
https://scholar.google.com/scholar?cites=7519589947873969922&as_sdt=2005&sciodt=0,5&hl=ko&start=0
22 4
https://scholar.google.com/scholar?cites=2514086522488325673&as_sdt=2005&sciodt=0,5&hl=ko&start=0
26 5
https://scholar.google.com/scholar?cites=16071254876387765388&as_sdt=2005&sciodt=0,5&hl=ko&start=0
35 6
https://scholar.google.com/scholar?cites=12916375234956339953&as_sdt=2005&sciodt=0,5&hl=ko&start=0
43 7
https://scholar.google.com/scholar?cites=17925254767584504435&as_sdt=2005&sciodt=0,5&hl=ko&start=0
46 8
https://scholar.google.com/scholar?cites=3520918811014873401&as_sdt=2005&sciodt=0,5&hl=ko&start=0
55 9
https://scholar.google.com/scholar?cites=14089872037077614590&as_sdt=2005&sciodt=0,5&hl=ko&start=0

In [6]:
df

,title,abstract,author,journal,year,pdf_link,keyword
0,Tests of significance in harmonic analysis,,RA Fisher,Proceedings of the Royal Society of London,1929,https://royalsocietypublishing.org/doi/abs/10....,NaN
0,Spectrum estimation and harmonic analysis,,DJ Thomson,"Proceedings of the IEEE,",1982,https://ieeexplore.ieee.org/abstract/document/...,NaN
0,Generalized harmonic analysis,,N Wiener,"Acta mathematica,",1930,https://link.springer.com/article/10.1007/BF02...,NaN
0,"Review of harmonic analysis, modeling and miti...",,"A Kalair, N Abas, AR Kalair, Z Saleem…",…,2017,https://www.sciencedirect.com/science/article/...,NaN
0,Harmonic analysis on hyperboloids,,RS Strichartz,"Journal of Functional Analysis,",1973,https://www.sciencedirect.com/science/article/...,NaN
...,...,...,...,...,...,...,...
0,Investigation on the pore structure and multif...,,"P Zhao, Z Wang, Z Sun, J Cai, L Wang","Marine and Petroleum Geology,",2017,https://www.sciencedirect.com/science/article/...,NaN
0,The utilization of selective laser melting tec...,,"D Jafari, WW Wits","Renewable and Sustainable Energy Reviews,",2018,https://www.sciencedirect.com/science/article/...,NaN
0,Tortuosity of porous media: Image analysis and...,,"J Fu, HR Thomas, C Li",E,arth,https://www.sciencedirect.com/science/article/...,NaN
0,Shear mechanical responses of sandstone expose...,,"Q Yin, J Wu, C Zhu, M He, Q Meng, H Jing",…,2021,https://link.springer.com/article/10.1007/s409...,NaN


In [7]:
df.to_csv('journal_1128.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     217 non-null    object
 1   abstract  217 non-null    object
 2   author    217 non-null    object
 3   journal   217 non-null    object
 4   year      217 non-null    object
 5   pdf_link  217 non-null    object
 6   keyword   0 non-null      object
dtypes: object(7)
memory usage: 13.6+ KB
